### Implementation, training and testing of a simple Recurrent Neural Network for time series forecasting

In this notebook, we will implement a simple recurrent neural network (RNN) from scratch using only Python and NumPy.

A key characteristic of the neural networks you have encountered so far, like densely connected networks and convolutional networks, is their lack of memory. They process each input independently, without retaining any state across inputs. To handle sequences or time-series data using such networks, you must present the entire sequence at once, treating it as a single data point. For instance, as you’re reading this text, your brain is processing its content word by word while keeping memories of what was read before. This way, you can have a fluid representation of the overall content.


RNNs adopt the same principle. Basically, RNNs are a type of artificial neural network designed to work with sequential data. Unlike traditional feedforward neural networks, RNNs have internal connections that loop back, allowing information to persist across time steps. This makes RNNs particularly suited for tasks such as time series prediction, natural language processing, and sequence classification. In practice, a single sequence of $N$ elements will be considered as a single input to the RNN. A standard DNN would take that input as an $N$ size input and process all the elements at once in a single step. What changes in an RNN is that this single input is no longer processed in a single step, but the RNN loops over sequence elements internally.

An RNN consists of the following components:

Input Layer $\rightarrow$ Processes the sequential data.

Hidden Layer $\rightarrow$ Contains neurons with internal connections to model temporal dependencies.

Output Layer $\rightarrow$ Produces predictions.

The output at a time $t$ is defined as
$$
y_t = \mathbf{W}_y\cdot s_t + b_y
$$
with $\mathbf{W}_y$ the matrix of weights for the output, $b_y$ the bias for the output and $s_t$ the internal state at time $t$, given by
$$
s_t = \tanh\left(\mathbf{W}_x \cdot x_t + \mathbf{W}_s \cdot s_{t-1} + b_s\right)
$$
where $x_t$ is the input at time t, $s_{t-1}$ is the internal state at time $t-1$, $\mathbf{W}_x$ is the matrix of weights for the input, $\mathbf{W}_s$ is the matrix of weights for the internal state and $b_s$ is the bias for the internal state. Here, we have assumed a $\tanh$ activation function, but other types of activation function are possible.

In this notebook, we will code and train a simple RNN to predict a shifted sequence given a sequence of $n$ previous steps, i.e.
$$
(x_{t-n}, x_{t-(n-1)},\dots,x_{t-1})\rightarrow RNN \rightarrow (x_{t-(n-1)},x_{t-(n-2)},\dots,x_{t})
$$

This is called sequence-to-sequence coding, it is a very simple case, but still essential for time series forecasting. Indeed, the prediction at $t$ provided by the RNN will be used as an input for the next step for the RNN to predict the output at $t+1$ and so on. This is called "iterative prediction", but in this Notebook we will focus only on the prediction of a single time step given a time sequence.

First of all, let us import the essential modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Now, we will generate synthetic data, using for example a sine wave. We will generate a given number of samples. Each sample will contain a sequence $x$ with a number of elements and a sequence $y$ with the same number of elements, equal to $x$ but shifted by one time step forward, with the same number of elements.

In [ ]:
def generate_sine_wave(seq_length, num_samples):
    X = []
    y = []
    for _ in range(num_samples):
        start = np.random.rand() * 2 * np.pi
        x_seq = np.sin(np.linspace(start, start + 2 * np.pi, seq_length + 1))
        ...................  # Input sequence
        ...................   # Target sequence
    return np.array(X), np.array(y)

Now we will use the previously coded function to generate the synthetic data

In [ ]:
seq_length = 10
num_samples = 1000
X, y = ...................
#Check the shape and length of the arrays
...................
...................
...................
...................

The next cell contains the class that defines the RNN. It contains in particular the forward method (given an $x$ sequence, predict the time-shifted $y$ sequence) and the backward method to update the trainable parameters using a simple gradient descent algorithm.

In [ ]:
# RNN Class
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.Wx = np.random.randn(hidden_size, input_size) * 0.01
        self.Ws = ...................
        self.Wy = ...................
        self.bs= np.zeros((..................., 1))
        self.by = np.zeros((..................., 1))

        self.intermediate_values = []  # Store intermediate values for backpropagation

    def forward(self, x_seq):
        s = np.zeros((self.hidden_size, 1))
        self.intermediate_values = []  # Clear the intermediate values for a new sequence
        y_pred = []

        for x_t in x_seq:
            x_t = x_t.reshape(-1, 1)  # Ensure x_t is a column vector
            s = ...................
            y_t = ...................
            y_pred.append(y_t)
            self.intermediate_values.append((s, x_t))  # Save internal state and input for backpropagation

        return np.array(y_pred).squeeze()

    # The next method computes the gradients of the loss function with respect to the
    # parameters of the model (Wx, Ws, Wy, bs, by) and updates them using gradient descent
    def backward(self, dLdy, learning_rate):
        dWx = np.zeros_like(self.Wx)
        dWs = np.zeros_like(self.Ws)
        dWy = np.zeros_like(self.Wy)
        dbs = np.zeros_like(self.bs)
        dby = np.zeros_like(self.by)
        ds_next = np.zeros((self.hidden_size, 1))

        # Loop Through Timesteps in Reverse
        # RNNs backpropagate through time, i.e.
        # it iterates backward through the sequence
        # to compute gradients at each time step
        for t in reversed(range(len(self.intermediate_values))):
            # Retrieve intermediate values values
            s, x_t = self.intermediate_values[t]
            
            # Compute gradient of output weights and biases
            # dLdy_t is the loss gradient for the output at time t
            dLdy_t = dLdy[t].reshape(-1, 1)  # Ensure dy is a column vector
            dWy += np.dot(dLdy_t, s.T)
            dby += dLdy_t
            
            #Backpropagate to the internal state
            ds = np.dot(self.Wy.T, dLdy_t) + ds_next
            # ds_tanh applies the derivative of the tanh activation function, (1−s^2)
            ds_tanh = ................... * ds

            # Compute gradients for the input and the internal state weights and biases
            dWx += np.dot(ds_tanh, x_t.T)
            dWs += np.dot(ds_tanh, self.intermediate_values[t - 1][0].T) if t > 0 else 0
            dbs += ds_tanh
            
            # Update the gradient for the next time step
            ds_next = np.dot(self.Ws.T, ds_tanh)

        # Update weights and biases
        self.Wx -= ...................
        self.Ws -= ...................
        self.Wy -= ...................
        self.bs -= ...................
        self.by -= ...................

The next step is the training of the RNN

In [ ]:
# Train the RNN
# Select a seed for the random generator just to ensure that the results can be reproduced (for debugging purposes)
np.random.seed(42)
#Initialize the RNN
...................

#Select the number of epochs
epochs = ...................

total_loss = []

#Select a learning rate
learning_rate = ...................

for epoch in range(epochs):
    epoch_loss = 0
    for i in range(len(X)):
        x_seq = X[i]
        y_true = y[i]

        # Complete the forward pass
        y_pred = ...................

        # Compute the loss using the mean equared error (MSE)
        loss = ...................
        
        #Add the loss to the epoch_loss
        ...................

        # Backward pass
        # Compute the gradient of the MSE loss with respect to the prediction y_pred
        ...................
        # Update the model parameters using the backward method of the RNN class
        ...................
    
    # Append the obtained loss to the total_loss list, normalizing by the number of samples
    ...................
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss[-1]:.4f}")

Let us now visualize the training loss as a function of the number of epochs

In [ ]:
# Visualize the Loss Curve
...................

Let us test our trained RNN on a single time sequence (num_samples=1)

In [ ]:
# Test the RNN
# Visualize predictions on a new sine wave.
...................

Plot the results...

In [ ]:
# Plot the input sequence and the predicted sequence.
# The predicted one is supposed to be the input sequence, but shifted by one time step forward
...................

If you have time, you can try to code the iterative prediction, which means that you use now the predicted value as an input for the RNN to predict the next step and so on. What do you observe?